Importar Librerias  

In [31]:
# gestión de datos
import numpy as np
import pandas as pd
import math

# fechas
import datetime as dt
from datetime import datetime
from dateutil.relativedelta import relativedelta

# gráficos
import seaborn as sns
import matplotlib.pyplot as plt
import jinja2

# preprocessing
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

# modelos
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

# metricas
import sklearn.metrics as metrics

# interpretabilidad
import shap

# hiperparametrizado
from sklearn.model_selection import GridSearchCV

# quitar alertas
import warnings
warnings.filterwarnings("ignore")

In [32]:
df = pd.read_csv('./DATA/pedidos_entrenar.csv')
df_pred = pd.read_csv("./DATA/pedidos_predecir.csv")

In [33]:
df.head(1)

,order_id,local_time,country_code,store_address,payment_status,n_of_products,products_total,final_status
0,33446280,14:11:09,AR,55379,PAID,2,11.88,DeliveredStatus


In [34]:
df["payment_status"].value_counts()

payment_status
PAID        53854
NOT_PAID      470
DELAYED         6
Name: count, dtype: int64

In [35]:
for i in df:
    print("Value_counts",df[i].value_counts(),"\n")
    print("Unique",df[i].unique(),"\n")

Value_counts order_id
33446280    1
32766208    1
32869927    1
32137313    1
32609872    1
           ..
32363605    1
33439367    1
32773775    1
32655367    1
32695191    1
Name: count, Length: 54330, dtype: int64 

Unique [33446280 33107339 32960645 ... 32895248 32792276 32695191] 

Value_counts local_time
09:03:01    38
10:03:06    31
10:03:05    26
10:03:01    25
10:03:02    24
            ..
18:16:53     1
21:36:02     1
11:42:25     1
09:42:03     1
12:21:30     1
Name: count, Length: 32905, dtype: int64 

Unique ['14:11:09' '11:47:41' '11:53:53' ... '23:12:31' '13:24:05' '12:21:30'] 

Value_counts country_code
AR    11854
ES    11554
TR     5696
PE     4284
UA     3729
IT     2537
EC     2265
RO     1957
FR     1911
EG     1643
MA     1446
CR     1000
CL      994
PA      909
PT      818
GT      511
GE      485
DO      448
UY      169
KE       84
PR       29
CI        6
BR        1
Name: count, dtype: int64 

Unique ['AR' 'GT' 'CR' 'ES' 'PE' 'PA' 'FR' 'IT' 'TR' 'EC' 'RO' 'KE' '

In [36]:
df.isnull().sum()

order_id          0
local_time        0
country_code      0
store_address     0
payment_status    0
n_of_products     0
products_total    0
final_status      0
dtype: int64

In [37]:
df_pred.isnull().sum()

order_id          0
local_time        0
country_code      0
store_address     0
payment_status    0
n_of_products     0
products_total    0
dtype: int64

In [38]:
df.duplicated().sum()

0

In [39]:
df_pred.duplicated().sum()

0

## pasamos Local_Time a fecha hora

In [40]:
df["local_time"] = pd.to_datetime(df["local_time"])

In [41]:
df["hora_compra"] = df["local_time"].dt.hour

In [42]:
df_pred["local_time"] = pd.to_datetime(df_pred["local_time"])

In [43]:
df_pred["hora_compra"] = df_pred["local_time"].dt.hour

In [44]:
df.nunique()

order_id          54330
local_time        32905
country_code         23
store_address      5627
payment_status        3
n_of_products        27
products_total     3904
final_status          2
hora_compra          24
dtype: int64

In [45]:
del(df["local_time"])
del(df_pred["local_time"])

In [46]:
df

,order_id,country_code,store_address,payment_status,n_of_products,products_total,final_status,hora_compra
0,33446280,AR,55379,PAID,2,11.88,DeliveredStatus,14
1,33107339,GT,23487,PAID,2,5.20,DeliveredStatus,11
2,32960645,CR,62229,PAID,1,6.03,DeliveredStatus,11
3,32089564,ES,29446,PAID,6,6.37,DeliveredStatus,20
4,32157739,AR,13917,PAID,1,5.36,CanceledStatus,21
...,...,...,...,...,...,...,...,...
54325,33443728,AR,63825,PAID,2,10.39,DeliveredStatus,13
54326,33499561,AR,27741,PAID,1,9.07,DeliveredStatus,17
54327,32895248,PA,65895,PAID,2,1.65,DeliveredStatus,20
54328,32792276,ES,20134,PAID,6,8.04,DeliveredStatus,13


### Ahora vamos a tratar la hora de compra no como una variable lineal sino como una variable ciclica. Esto es porque si la tratamos de forma lineal el modelo puede pensar que el las 12 esta en extremos opuesto que la 1...y no es así, son contiguas


In [47]:
# pasamos la variable ciclica a 2 variables con su seno y coseno
df['hora_compra'] = df['hora_compra'] * 2 * math.pi / 24
df_pred['hora_compra'] = df_pred['hora_compra'] * 2 * math.pi / 24

df['hora_compra_cos'] = df['hora_compra'].apply(lambda x: math.cos(x))
df_pred['hora_comprae_cos'] = df_pred['hora_compra'].apply(lambda x: math.cos(x))
df['hora_compra_sin'] = df['hora_compra'].apply(lambda x: math.sin(x))
df_pred['hora_compra_sin'] = df_pred['hora_compra'].apply(lambda x: math.sin(x))

df = df.drop('hora_compra', axis=True)
df_pred = df_pred.drop('hora_compra', axis=True)

In [48]:
df

,order_id,country_code,store_address,payment_status,n_of_products,products_total,final_status,hora_compra_cos,hora_compra_sin
0,33446280,AR,55379,PAID,2,11.88,DeliveredStatus,-0.866025,-5.000000e-01
1,33107339,GT,23487,PAID,2,5.20,DeliveredStatus,-0.965926,2.588190e-01
2,32960645,CR,62229,PAID,1,6.03,DeliveredStatus,-0.965926,2.588190e-01
3,32089564,ES,29446,PAID,6,6.37,DeliveredStatus,0.500000,-8.660254e-01
4,32157739,AR,13917,PAID,1,5.36,CanceledStatus,0.707107,-7.071068e-01
...,...,...,...,...,...,...,...,...,...
54325,33443728,AR,63825,PAID,2,10.39,DeliveredStatus,-0.965926,-2.588190e-01
54326,33499561,AR,27741,PAID,1,9.07,DeliveredStatus,-0.258819,-9.659258e-01
54327,32895248,PA,65895,PAID,2,1.65,DeliveredStatus,0.500000,-8.660254e-01
54328,32792276,ES,20134,PAID,6,8.04,DeliveredStatus,-0.965926,-2.588190e-01


## Categoricas

In [49]:
df["final_status"] = np.where(df["final_status"] == "DeliveredStatus", 1, 0) #Cambiamos la variable target a 1 si esta entregado y 0 si no se entrega

In [50]:
# Tb podríamos hacerlo asi :
#df['final_status'] = df['final_status'].apply(lambda x: 1 if x == 'DeliveredStatus'  else 0)

In [51]:
df["final_status"] 


0        1
1        1
2        1
3        1
4        0
        ..
54325    1
54326    1
54327    1
54328    1
54329    1
Name: final_status, Length: 54330, dtype: int32